In [192]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', None)

<h2 style="color:#1f77b4; font-weight:bold;">Veri Setlerinin İçe Aktarılması</h2>

Bu çalışmada hanehalkı anket verileri ile mekansal analizler yapılabilmesi için çeşitli veri kaynakları içe aktarılmıştır. Bu veri setleri, hem sosyo-demografik bilgiler hem de coğrafi bölgeleme (TAZ) bilgilerini kapsamaktadır.

---

### 📥 **Anket Veri Setlerinin Yüklenmesi**

Aşağıda üç temel Excel dosyası içe aktarılmıştır:

- **Profil Verisi (`profile`)**: Hanehalkı bireylerine ait yaş, eğitim, çalışma durumu gibi demografik bilgiler içerir.  
- **Hane Bilgisi Verisi (`house`)**: Her bir haneye ait konum bilgileri, gelir grubu ve temel yapı özellikleri yer alır.  
- **Araç Sahipliği Verisi (`vehicle`)**: Hanelerin sahip olduğu araç türü, sayısı ve araç tipi gibi bilgileri içerir.

---

### 🌍 **TAZ (Traffic Analysis Zones) Verisinin Yüklenmesi**

`TAZ` verisi, trafik analiz bölgelerini tanımlayan bir **shapefile** dosyasıdır ve analizlerin mekansal temelini oluşturur. Bu veri, tüm analizlerde bölgesel karşılaştırma ve toplulaştırma amacıyla kullanılacaktır.
 kullanılacaktır.


In [194]:
# Profil verisi
profile = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\TAZ_Analyis\\Data\\HTS\\Profile.xlsx')
# Hane bilgisi verisi
house = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\TAZ_Analyis\\Data\\HTS\\Household.xlsx')
# Araç sahipliği verisi
vehicle = pd.read_excel('C:\\Users\\Paraboly\\Desktop\\TAZ_Analyis\\Data\\HTS\\Vehicle_ownership.xlsx')
taz = gpd.read_file('C:\\Users\\Paraboly\\Desktop\\TAZ_Analyis\\Data\\SHP\\TAZs_zone.shp')

In [195]:
taz.head(1)

,NO,CODE,geometry
0,1,None,"MULTIPOLYGON (((666259.641 4526986.655, 666259..."


In [196]:
profile.head(1)

,p_id,hh_id,participation_status,person_order,name,age,gender,District,edu,edu_status,drive_lic,house_rep_prox,work_status,work_type,sector,job,work_time,office_info,work_days,pt_card,pt_card_type,disability,had_trip,no_trip_reason
0,71,6kaqzPZwvLgUnuUalhTx,Evet,K1,Şükran Çiftçi,64.0,Kadın,BEYKOZ,İlköğretim mezunu,Öğrenci Değil,Yok,Kendisi,Çalışmıyor,NaN,NaN,NaN,NaN,NaN,0.0,Yok,NaN,Hayır,Hayır,Gerek Olmaması


In [197]:
house.head(1)

,Unnamed: 0,hh_id,survey_id,hh_type,district,neig,street,apartment_no,door_no,house_type,sample_type,tuik_id,latitude,longitude,hh_size,hh_has_vehicle,hh_vehicle_count,residence_status,residence_status_other,rental_value,hh_income
0,0,hbg0qx2dfppdajxfSIit,hbg0qx2dfppdajxfSIit,Anne ve çocuklardan oluşan,ŞİLE,MEŞRUTİYET,KASIMAĞA SOKAK,4,2,Müstakil Konut,Asil,A4-8142-5,41.15871,29.562484,3,Evet,1,Ev sahibi,NaN,NaN,60.001 - 80.000 TL


In [198]:
vehicle.head(1)

,hh_id,v_id,type,brand,model,year,fuel,bike_type,ownership_status,autopark,most_user
0,sjmlC3dl4rNI9nIuWQ9Q,14610,Binek Otomobil,Opel,astra,2011.0,Benzin&LPG,NaN,Hane,Yol Kenarı-ücretsiz,K2


<h2 style="color:#1f77b4; font-weight:bold;">🔍 Veri İşleme ve Analiz Süreci</h2>

Bu bölümde, içe aktarılan anket ve mekansal veriler üzerinde çeşitli ön işleme ve analiz adımları gerçekleştirilecektir. Amaç, hanehalkı verilerini TAZ poligonlarıyla mekansal olarak eşleştirerek **nüfus**, **öğrenci oranı**, **gelir dağılımı** gibi göstergeleri bölgesel düzeyde analiz etmektir.

Aşağıdaki işlemler sırasıyla gerçekleştirilecektir:

- Koordinat sisteminin standardize edilmesi (EPSG:4326)
- Noktasal verilerin TAZ poligonları ile eşleştirilmesi (spatial join)
- TAZ bazında nüfus ve öğrenci sayısının hesaplanması
- Gelir gruplarının sınıflandırılması ve bölgesel dağılımının çıkarılması


<h3 style="font-weight:bold;">Koordinat Referans Sisteminin (CRS) EPSG:4326 (WGS 84) Olarak Ayarlanması ve Kontrolü</h3>


In [199]:
# CRS'yi EPSG:4326 (WGS 84) ve kontrolü
taz = taz.to_crs(epsg=4326)
taz.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [200]:
taz.head(1)

,NO,CODE,geometry
0,1,None,"MULTIPOLYGON (((28.97317 40.87709, 28.97317 40..."


<h3 style="font-weight:bold;">TAZ Bazlı Nüfus Verisinin Hesaplanması <code>HTS_POP</code></h3>

Bu adımda, her bir TAZ (Traffic Analysis Zone) poligonunun içinde kalan kişi sayısı hesaplanmıştır. `merged_profile` veri setinde yer alan bireylerin konum bilgileri (latitude-longitude) kullanılarak, bu noktaların hangi TAZ sınırları içerisinde yer aldığı belirlenmiştir.

Ardından, her TAZ bölgesine düşen toplam kişi sayısı sayılarak, `taz` GeoDataFrame'ine <code>HTS_POP</code> adlı yeni bir kolon olarak eklenmiştir.

**<code>HTS_POP</code>:**  
Her TAZ poligonunun içinde kalan kişi sayısını ifade eder. Yani, ilgili TAZ içinde kaç kişi bulunduğunu gösterir.

Bu veri, mekansal nüfus yoğunluğu analizleri ve bölgesel karşılaştırmalar için temel bir göstergedir.


In [201]:
merged_profile= profile.merge(house, on='hh_id', how='left')

In [202]:
print("merged_profile tablosundaki kişi sayısı:", merged_profile['hh_id'].count())

merged_profile tablosundaki kişi sayısı: 169423


In [203]:
missing_coords = merged_profile[merged_profile['latitude'].isna() | merged_profile['longitude'].isna()]
print("Eksik koordinat sayısı:", len(missing_coords))

Eksik koordinat sayısı: 0


In [204]:
both_zero_coords = merged_profile[(merged_profile['latitude'] == 0) & (merged_profile['longitude'] == 0)]
print("Hem latitude hem de longitude sıfır olan satır sayısı:", len(both_zero_coords))

Hem latitude hem de longitude sıfır olan satır sayısı: 0


In [205]:
import geopandas as gpd
from shapely.geometry import Point
# Nokta geometrilerini oluştur
merged_profile['geometry'] = merged_profile.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# GeoDataFrame'e çevir (aynı CRS - WGS84)
merged_profile = gpd.GeoDataFrame(merged_profile, geometry='geometry', crs='EPSG:4326')
merged_profile.head(1)

,p_id,hh_id,participation_status,person_order,name,age,gender,District,edu,edu_status,drive_lic,house_rep_prox,work_status,work_type,sector,job,work_time,office_info,work_days,pt_card,pt_card_type,disability,had_trip,no_trip_reason,Unnamed: 0,survey_id,hh_type,district,neig,street,apartment_no,door_no,house_type,sample_type,tuik_id,latitude,longitude,hh_size,hh_has_vehicle,hh_vehicle_count,residence_status,residence_status_other,rental_value,hh_income,geometry
0,71,6kaqzPZwvLgUnuUalhTx,Evet,K1,Şükran Çiftçi,64.0,Kadın,BEYKOZ,İlköğretim mezunu,Öğrenci Değil,Yok,Kendisi,Çalışmıyor,NaN,NaN,NaN,NaN,NaN,0.0,Yok,NaN,Hayır,Hayır,Gerek Olmaması,59878,6kaqzPZwvLgUnuUalhTx,"Anne, Baba ve çocuklardan oluşan",BEYKOZ,KAVACIK,ÖGE SOKAK,9,4,Apartman Dairesi,Asil,AS-2520-2,41.094734,29.082197,4,Hayır,0,Ev sahibi,NaN,NaN,15.001 - 25.000 TL,POINT (29.08220 41.09473)


In [206]:
# 2. Spatial Join – Hangi nokta hangi TAZ içinde?
points_in_taz = gpd.sjoin(merged_profile, taz, how='inner', predicate='within')

# 3. Her TAZ poligonuna düşen nokta sayısını hesapla
taz_counts = points_in_taz.groupby('index_right').size().reset_index(name='HTS_POP')

# 4. HTS_POP kolonunu TAZ GeoDataFrame'e ekle
taz['HTS_POP'] = taz_counts.set_index('index_right')['HTS_POP']

# 5. Veri olmayanları 0 yap
taz['HTS_POP'] = taz['HTS_POP'].fillna(0).astype(int)

In [207]:
taz.head(1)

,NO,CODE,geometry,HTS_POP
0,1,None,"MULTIPOLYGON (((28.97317 40.87709, 28.97317 40...",17


In [208]:
taz['HTS_POP'].sum()

169423

In [238]:
# ✅ Check: HTS_POP toplamı ile profile tablosundaki kişi sayısı sayısı eşit mi?
check_result = taz['HTS_POP'].sum() - merged_profile['hh_id'].count()

if check_result == 0:
    print("✔️ Doğrulama başarılı: HTS_POP toplamı ile kişi sayısı eşit.")
else:
    print(f"❌ Doğrulama hatalı: Fark = {check_result}")


✔️ Doğrulama başarılı: HTS_POP toplamı ile kişi sayısı eşit.


<h3 style="font-weight:bold;">TAZ bazlı Öğrenci Sayılarının Eklenmesi <code>HTS_EDU</code></h3>

Bu adımda, `merged_profile` veri setinde yer alan ve <code>edu_status</code> değeriri "Öğrenci" olan bireyler filtrelenmiştir. Her öğrencinin konum bilgisi (latitude-longitude) kullanılarak, hangi TAZ poligonuna denk geldiği belirlenmiştir.

Spatial join (mekansal birleştirme) işlemi ile her öğrencinin ait olduğu TAZ bulunmuş ve her TAZ için düşen toplam öğrenci sayısı hesaplanarak `HTS_EDU` adlı yeni bir kolon olarak `taz` veri setine eklenmiştir.

**<code>HTS_EDU</code>:**  
Her TAZ poligonunun içinde kalan "öğrenci" bireylerin sayısını ifadabmemiştir.


In [209]:
# 1. Öğrencileri filtrele
students = merged_profile[merged_profile['edu_status'] == 'Öğrenci'].copy()
#öğrenci sayısı 

print("Profile tablosundaki öğrenci sayısı:", students['edu_status'].count())


Profile tablosundaki öğrenci sayısı: 29191


In [210]:
#spatial join yapılamayacak veri sayısı
both_zero_coords_2 = students[(students['latitude'] == 0) & (students['longitude'] == 0)]
print("Hem latitude hem de longitude sıfır olan satır sayısı:", len(both_zero_coords_2))


Hem latitude hem de longitude sıfır olan satır sayısı: 0


In [211]:
# Öğrencilerin TAZ poligonları içine düşenleri spatial join ile eşle
joined_students = gpd.sjoin(students, taz, how='inner', predicate='within')

#Her TAZ (index_right) için öğrenci sayısını say
edu_counts = joined_students['index_right'].value_counts().rename_axis('index').reset_index(name='HTS_EDU')

#TAZ veri setine HTS_EDU kolonunu ekle
taz['HTS_EDU'] = 0  # Varsayılan değer
taz.loc[edu_counts['index'], 'HTS_EDU'] = edu_counts['HTS_EDU'].values

#Spatial join yapılan öğrenci sayısı
taz['HTS_EDU'].sum()

29191

In [239]:
# ✅ Check: HTS_EDU toplamı ile profile tablosundaki öğrenci sayısı sayısı eşit mi?
check_result = taz['HTS_EDU'].sum() - students['edu_status'].count()

if check_result == 0:
    print("✔️ Doğrulama başarılı: HTS_EDU toplamı ile öğrenci sayısı eşit.")
else:
    print(f"❌ Doğrulama hatalı: Fark = {check_result}")


✔️ Doğrulama başarılı: HTS_EDU toplamı ile öğrenci sayısı eşit.


<h3 style="font-weight:bold;">TAZ Bazlı Hane Gelirlerinin Eklenmesi <code>INC_*</code></h3>

Bu adımda, `house` veri setindeki her hanenin gelir bilgisi (`income_class`) kullanılarak TAZ poligonları ile eşleştirme yapılmıştır. Spatial join (mekansal birleştirme) işlemiyle her hanenin bulunduğu TAZ belirlenmiş ve TAZ bazında her gelir sınıfına ait hane sayısı hesaplanmıştır.

Her gelir sınıfı, ayrı bir kolon olarak `taz` GeoDataFrame’ine eklenmiştir.

#### ➤ Oluşan Gelir Grupları ve Anlamları:

- (<code>INC_0_5K</code>) → 0 - 5.000 TL arası gelir grubu  
- (<code>INC_5K_10K</code>) → 5.001 - 10.000 TL arası gelir grubu  
- (<code>INC_10K_15K</code>) → 10.001 - 15.000 TL arası gelir grubu  
- (<code>INC_15K_25K</code>) → 15.001 - 25.000 TL arası gelir grubu  
- (<code>INC_25K_40K</code>) → 25.001 - 40.000 TL arası gelir grubu  
- (<code>INC_40K_60K</code>) → 40.001 - 60.000 TL arası gelir grubu  
- (<code>INC_60K_80K</code>) → 60.001 - 80.000 TL arası gelir grubu  
- (<code>INC_80K_100K</code>) → 80.001 - 100.000 TL arası gelir grubu  
- (<code>INC_100K_140K</code>) → 100.001 - 140.000 TL arası gelir grubu  
- (<code>INC_140K_200K</code>) → 140.001 - 200.000 TL arası gelir grubu  
- (<code>INC_200KPLUS</code>) → 200.001 TL ve üzeri gelir grubu  
- (<code>INC_Unknown</code>) → Gelir bilgisini belirtmeyen veya earla kullanılabilir.
sik haneler

In [212]:
house['hh_income'].unique()

array(['60.001 - 80.000 TL', '15.001 - 25.000 TL', '40.001 - 60.000 TL',
       '25.001 - 40.000 TL', '80.001 - 100.000 TL',
       '100.001 - 140.000 TL', '10.001 - 15.000 TL',
       '200.001TL’den fazla', 'Belirtmek istemiyorum.',
       '140.001 - 200.000 TL', '60.001 - 100.000 TL', '5.001 – 10.000 TL',
       '5.000 TL’den az', '100.000TL’den fazla'], dtype=object)

In [213]:
print("household tablosundaki gelir bilgisi olan toplam hane sayısı:", house['hh_income'].count())


household tablosundaki gelir bilgisi olan toplam hane sayısı: 59879


In [214]:
#spatial join yapılamayacak veri sayısı
both_zero_coords_3 = house[(house['latitude'] == 0) & (house['longitude'] == 0)]
print("Hem latitude hem de longitude sıfır olan satır sayısı:", len(both_zero_coords_3))


Hem latitude hem de longitude sıfır olan satır sayısı: 0


In [215]:
def classify_income(group):
    if group == '5.000 TL’den az':
        return '0-5K'
    elif group == '5.001 – 10.000 TL':
        return '5K–10K'
    elif group == '10.001 - 15.000 TL':
        return '10K–15K'
    elif group == '15.001 - 25.000 TL':
        return '15K–25K'
    elif group == '25.001 - 40.000 TL':
        return '25K–40K'
    elif group == '40.001 - 60.000 TL':
        return '40K–60K'
    elif group == '60.001 - 80.000 TL':
        return '60K–80K'
    elif group == '60.001 - 100.000 TL' or group == '80.001 - 100.000 TL':
        return '80K–100K'
    elif group == '100.001 - 140.000 TL':
        return '100K–140K'
    elif group == '140.001 - 200.000 TL':
        return '140K–200K'
    elif group in ['200.001TL’den fazla', '100.000TL’den fazla']:
        return '200K+'
    elif group == 'Belirtmek istemiyorum.':
        return 'Unknown'
    else:
        return 'Unknown'
        
# Yeni sütun oluştur
house['income_class'] = house['hh_income'].apply(classify_income)


In [216]:
# 3. GeoDataFrame'e çevir
house['geometry'] = house.apply(lambda r: Point(r['longitude'], r['latitude']), axis=1)
house = gpd.GeoDataFrame(house, geometry='geometry', crs='EPSG:4326')

# 4. TAZ ile spatial join
joined = gpd.sjoin(house, taz, how='inner', predicate='within')

# 5. TAZ bazında gelir sınıfına göre hane sayısı
income_counts = joined.groupby(['index_right', 'income_class']).size().unstack(fill_value=0)

# 6. Kolon adlarını düzenle
income_counts.columns = ['INC_' + col.replace('-', '_').replace('–', '_').replace('+', 'PLUS').replace('K', 'K') for col in income_counts.columns]

# 7. TAZ’a join et
taz = taz.join(income_counts, how='left')
taz.fillna(0, inplace=True)
taz[income_counts.columns] = taz[income_counts.columns].astype(int)

In [217]:
taz.head(1)

,NO,CODE,geometry,HTS_POP,HTS_EDU,INC_0_5K,INC_100K_140K,INC_10K_15K,INC_140K_200K,INC_15K_25K,INC_200KPLUS,INC_25K_40K,INC_40K_60K,INC_5K_10K,INC_60K_80K,INC_80K_100K,INC_Unknown
0,1,0,"MULTIPOLYGON (((28.97317 40.87709, 28.97317 40...",17,2,0,0,3,0,0,0,3,2,0,0,0,0


In [218]:
income_cols = [
    'INC_0_5K', 'INC_5K_10K', 'INC_10K_15K', 'INC_15K_25K', 'INC_25K_40K',
    'INC_40K_60K', 'INC_60K_80K', 'INC_80K_100K', 'INC_100K_140K',
    'INC_140K_200K', 'INC_200KPLUS', 'INC_Unknown'
]

# Toplamı hesapla
total_households = taz[income_cols].sum().sum()

print("Gelir grubu kolonlarındaki toplam hane sayısı:", total_households)

Gelir grubu kolonlarındaki toplam hane sayısı: 59879


In [240]:
# ✅ Check: İncome toplamı ile household tablosundaki income sayısı sayısı eşit mi?
check_result = total_households - house['hh_income'].count()

if check_result == 0:
    print("✔️ Doğrulama başarılı: INC* toplamı ile household income sayısı eşit.")
else:
    print(f"❌ Doğrulama hatalı: Fark = {check_result}")


✔️ Doğrulama başarılı: INC* toplamı ile household income sayısı eşit.
